クロスデバイス　Valis拡張との併用

In [20]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

476	ソニーネットワークコミュニケーションズ株式会社

In [21]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

190315


In [22]:
#デバイス
seed_sp_device = 'iOS','Android'
xd_sp_device = 'iphone','android_phone','windows_phone'

#ターゲット
advertiser_id = 476
#conversion_group_id = 

#grp = 'cvg'
#grp = 'rtg'

#クロスデバイステーブル
xd_table = 'segment.drawbridge_device_1901'

In [24]:
file_name = f"valis_Ad476_sp_lockon_RQq2zPub_5_190312"
print(file_name)

valis_Ad476_sp_lockon_RQq2zPub_5_190312


In [25]:
seed_table = f"dm_tmp.ya_seed_{file_name}"
print(seed_table)

dm_tmp.ya_seed_valis_Ad476_sp_lockon_RQq2zPub_5_190312


In [26]:
uid_table = f"dm_tmp.ya_xd_{file_name}"
print(uid_table)

dm_tmp.ya_xd_valis_Ad476_sp_lockon_RQq2zPub_5_190312


SP種ファイルを読込し、PCユーザー拡張セグメントを作成する

In [29]:
df = pd.read_csv(f"/home/yusuke_akada/notebook/{file_name}.tsv",header=None, names=['smn_uid'])
df.head()

,smn_uid
0,786000016873456
1,1067000010947933
2,936000123104261
3,1601000216241433
4,1535000001359561


In [30]:
df.shape[0]

329

In [31]:
#HC3 CSVをUP
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    ir.upload_dataframe(seed_table, df, ['bigint'])

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:Create empty 

In [32]:
%%slack_notify @yusuke_akada
#XD拡張

#f-string
query = (
    f"""
    select 
    distinct xd2.smn_uid
    from {seed_table} sd
    left join {xd_table} xd
    on sd.smn_uid = xd.smn_uid
    left join {xd_table} xd2
    on xd.drawbridge_consumer_id = xd2.drawbridge_consumer_id
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_fix = ir.sql_to_pandas(query)
    #ImpalaにUp
    #ir.upload_dataframe(uid_table, df_fix, ['bigint']) #Impala upは不要かも
    #テーブル削除
    ir.drop_tables([seed_table])
    
df_fix.to_csv(f'{uid_table}.tsv', sep='\t', index=False, header=False)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [33]:
df_fix.shape[0]

0